In [1]:
!pip install -q transformers --upgrade

ERROR: refractml 1.0.3 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: refractml 1.0.3 has requirement urllib3==1.26.15, but you'll have urllib3 2.1.0 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: google-auth 2.22.0 has requirement urllib3<2.0, but you'll have urllib3 2.1.0 which is incompatible.


In [1]:
import transformers

In [4]:
from transformers import AutoModelWithLMHead

In [5]:
from transformers import list_adapters

ImportError: cannot import name 'list_adapters' from 'transformers' (/tmp/pip_packages/transformers/__init__.py)